## Collaborative Filtering

Collaborative filtering merupakan teknik yang digunakan untuk memberikan rekomendasi mengenai minat pengguna dengan mengumpulkan preferensi atau informasi dari pengguna lainnya (kolaborasi). </br>

Pada project ini, rekomendasi yang akan diberikan adalah **rekomendasi film kepada seseorang berdasarkan review yang diberikan oleh pengguna lainnya**. Data yang digunakan pada project ini merupakan data mengenai 10 film yakni :
1. Ada Apa dengan Cinta 2
2. Gundala
3. Dilan 1991
4. Bumi Manusia
5. Dua Garis Biru
6. Avengers: End Game
7. The Lion King
8. Aladdin
9. Spiderman: Far From Home
10. Captain Marvel </br>

Jumlah pengguna yang digunakan pada project ini adalah 24 pengguna. Masing-masing pengguna memberikan rating terhadap film yang telah ditonton dengan memberikan rate mulai 1-5. Jika pengguna belum menonton film tersebut maka pengguna tidak memberikan rating terhadap film tersebut sehingga selanjutnya akan diberikan rekomendasi berdasarkan review dari pengguna lainnya.

**Import Libraries**

In [45]:
from math import sqrt
import pandas as pd
import numpy as np

Data yang digunakan berbentuk JSON sehingga dataset yang ada pada data ratingfilm akan diimpor untuk memudahkan analisis

In [46]:
from ratingfilm import dataset

### Similarity Score

Similarity score digunakan untuk mengetahui kemiripan antara dua item. Jika nilai kemiripannya mendekati 1 maka antar kedua item memiliki kemiripan, sedangkan jika nilai kemiripannya mendekati angka 0 maka antar kedua item tidak memiliki kemiripan.

In [47]:
def similarity_score(person1,person2):

    # Melihat jarak euclidean antara person 1 dan 2

    # Untuk mendapatkan penilaian item dari person 1 dan 2
    both_viewed = {}

    for item in dataset[person1]:
        if item in dataset[person2]:
            both_viewed[item] = 1
        
        # Keadaan ketika mereka memiliki kesamaan rating 
        if len(both_viewed) == 0:
            return 0

        # Menghitung jarak euclidean
        sum_of_eclidean_distance = []

        for item in dataset[person1]:
            if item in dataset[person2]:
                sum_of_eclidean_distance.append(pow(dataset[person1][item] - dataset[person2][item], 2))
        sum_of_eclidean_distance = sum(sum_of_eclidean_distance)
        
        return 1/(1+sqrt(sum_of_eclidean_distance))

Pada project ini akan mencari rekomendasi film untuk Rima. Sehingga akan dilakukan perhitungan kemiripan item dengan pengguna lainnya yakni Jawaharal.

In [48]:
print(similarity_score('Rima','Jawaharal'))

0.14459058185587106


Kesamaan antara Rima dan Jawaharal dalam memberikan rating pada setiap film adalah 0.145, hal ini menunjukkan bahwa **antara Rima dan Jawaharal tidak memiliki kemiripan dalam memberikan penilaian terhadap rating pada film.**

### Person Correlation

Person correlation menunjukkan apakah antar user memiliki kesamaan selera film atau tidak.Nilai person correlation berkisar antara -1 hingga 1 dimana korelasi person -1 menunjukkan bahwa antar kedua user memiliki selera film yang bertolak belakang sedangkan jika korelasi person 1 menunjukkan kedua user memiliki selera film yang sama

In [49]:
def person_correlation(person1, person2):

   # Mendapatkan item yang akan dinilai
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    # Mengecek Kesamaan Rating
    if number_of_ratings == 0:
        return 0

    # Menambahkan preferensi semua user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    #Menjumlahkan hasil kuadrat dari preferensi semua user 
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Menjumlahkan hasil kali dari preferensi masing-masing item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Menghitung nilai pearson
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

In [50]:
person_correlation('Dpv', 'luck')

0.5915747820033221

nilai person correlation antara Rima dan Jawaharal menunjukkan nilai 0.77 dimana hal ini menunjukkan bahwa **Dpv dan luck memiliki kesamaan dalam selera film.** dikarenakan nilai person correlation telah berada diatas 0,5

### Most Similar User

Most similar user menunjukkan urutan antara satu user dengan semua user dan diurutkan dari kesamaan selera film yang tertinggi

In [51]:
def most_similar_users(person, number_of_users):

    # returns the number_of_users (similar persons) for a given specific person
    scores = [(person_correlation(person, other_person), other_person) for other_person in dataset if other_person != person]

    # Sort the similar persons so the highest scores person will appear at the first
    scores.sort()
    scores.reverse()
    return scores[0:number_of_users]

In [52]:
print(most_similar_users('Dpv', 23))

[(0.7293051654988714, 'Rima'), (0.7165918706113663, 'Genjeh'), (0.7165918706113663, 'Febi ganteng gak ada obat'), (0.6220880993971143, 'Damar Teman Firli'), (0.5915747820033221, 'luck'), (0.583111187267418, 'Putrisqiana'), (0.5053114692007453, 'OM INDRA'), (0.4691808711301112, 'AhokTemanFirli'), (0.4378206416626331, 'Nonton_Saat_Diskon'), (0.36714408087207806, 'Romantika'), (0.2875397711492321, 'bunga'), (0.20908664397176108, 'Topik Zulkarnain'), (0.17277368511627203, 'Indra 1991 SM'), (0.1417910447761193, 'Star'), (0.12242264554887113, 'Jawaharal'), (0.10246640853342608, 'Mulya'), (0.0816602150356523, 'ANI'), (0.05120571267041111, 'Indra Junior'), (-0.03042374952996796, 'faizah'), (-0.04319342127906763, 'franadek'), (-0.05655346358155575, 'jul'), (-0.10538115744284798, 'Hania'), (-0.6000952607769756, 'Maria O.')]


Dari most similarity user diketahui jika **Dpv memiliki kesamaan selera film paling dekat dengan Rima.**

### Rekomendasi Film untuk Dpv

In [53]:
def user_recommendations(person):

    # Gets recommendations for a person by using a weighted average of every other user's rankings
    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        # don't compare me to myself
        if other == person:
            continue
        sim = person_correlation(person,other)
        #print ">>>>>>>",sim

        # ignore scores of zero or lower
        if sim <=0: 
            continue
        for item in dataset[other]:

            # only score movies i haven't seen yet
            if item not in dataset[person] or dataset[person][item] == 0:

            # Similrity * score
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                # sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+= sim

        # Create the normalized list

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    # returns the recommended items
    recommendataions_list = [recommend_item for score,recommend_item in rankings]
    return recommendataions_list

In [54]:
print(user_recommendations('Dpv'))

['Aladdin', 'The Lion King', 'Dua Garis Biru', 'Bumi Manusia']


### Conclusion

Dengan menggunakan collaborative filtering didapatkan hasil :
* Dpv memiliki kesamaan selera film paling dekat dengan Rima
* Film yang direkomendasikan untuk Devita berdasarkan review yang diberikan oleh Pengguna lain berturut-turut adalah :
    1. Aladdin
    2. The Lion King
    3. Dua Garis Biru
    4. Bumi Manusia